In [ ]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("BenchmarkTools")
Pkg.add("Tables")
Pkg.add("Statistics")
Pkg.add("Query")
Pkg.add("CUDA")
Pkg.add("Dates")
Pkg.add("Printf")

In [ ]:
Pkg.add("Parquet")

In [1]:
using CSV
using DataFrames
using BenchmarkTools
using Tables
using Statistics
using Query
using CUDA
using Dates
using Printf
using Parquet

In [ ]:
dfr = CSV.read("ratings.csv", DataFrame)

In [3]:
dfm = CSV.read("movies.csv", DataFrame)

Row,movieId,title,genres
,Int64,String,String
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,5,Father of the Bride Part II (1995),Comedy
6,6,Heat (1995),Action|Crime|Thriller
7,7,Sabrina (1995),Comedy|Romance
8,8,Tom and Huck (1995),Adventure|Children
9,9,Sudden Death (1995),Action


In [2]:
nF = 10

10

In [3]:
kg = [ "Action","Adventure","Animation","Children’s","Comedy","Crime","Documentary",
"Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller",
"War","Western","(no genres listed)"]

19-element Vector{String}:
 "Action"
 "Adventure"
 "Animation"
 "Children’s"
 "Comedy"
 "Crime"
 "Documentary"
 "Drama"
 "Fantasy"
 "Film-Noir"
 "Horror"
 "Musical"
 "Mystery"
 "Romance"
 "Sci-Fi"
 "Thriller"
 "War"
 "Western"
 "(no genres listed)"

In [4]:
ng = length(kg)

19

In [5]:
ra = zeros(Int, ng, nF)

19×10 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0

In [6]:
# CSV a Parquet
Parquet.write_parquet("movies.parquet",dfm)

LoadError: UndefVarError: `dfm` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
dfm_2 = DataFrame(read_parquet("movies.parquet"))
dfm_2 = dfm_2[:, [:movieId, :genres]]

Row,movieId,genres
,Int64?,String?
1,1,Adventure|Animation|Children|Comedy|Fantasy
2,2,Adventure|Children|Fantasy
3,3,Comedy|Romance
4,4,Comedy|Drama|Romance
5,5,Comedy
6,6,Action|Crime|Thriller
7,7,Comedy|Romance
8,8,Adventure|Children
9,9,Action


In [8]:
# CSV a Parquet Ratings
Parquet.write_parquet("ratings.parquet",dfr)

LoadError: UndefVarError: `dfr` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [9]:
function FindRatingsWorker(index, ng, kg, dfm_2, dfr_v)
    # Inicializar el contador para este archivo de ratings
    genre_count = zeros(Int, ng)
    
    # Unir el DataFrame de ratings con el DataFrame de películas
    data = leftjoin(dfr_v, dfm_2, on = :movie_id)
    
    # Contar los géneros en este archivo
    for row in eachrow(data)
        if !ismissing(row.genres)
            genre_index = findfirst(x -> x == row.genres, kg)
            if genre_index !== nothing
                genre_count[genre_index] += 1
            end
        end
    end
    
    return genre_count
end

FindRatingsWorker (generic function with 1 method)

In [ ]:
dfr_v = DataFrame(read_parquet("ratings.parquet"))

In [21]:
Threads.@threads for i in 1:nF
    # Generar el nombre del archivo de ratings
    rfn = "dfr" * lpad(i, 2, '0') * ".parquet"  # Ajusta el nombre según tus archivos
    
    # Cargar el archivo de ratings
    dfr_v = DataFrame(read_parquet(rfn))
    
    # Llamar a la función que procesa los ratings y actualiza el conteo por género
    ra[:, i] = FindRatingsWorker(i, ng, kg, dfm_2, dfr_v)
end

LoadError: TaskFailedException

[91m    nested task error: [39mInvalid path to parquet file or dataset - dfr01.parquet
    Stacktrace:
     [1] [0m[1merror[22m[0m[1m([22m[90ms[39m::[0mString[0m[1m)[22m
    [90m   @[39m [90mBase[39m [90m./[39m[90m[4merror.jl:35[24m[39m
     [2] [0m[1mread_parquet[22m[0m[1m([22m[90mpath[39m::[0mString; [90mkwargs[39m::[0m@Kwargs[90m{}[39m[0m[1m)[22m
    [90m   @[39m [35mParquet[39m [90m~/.julia/packages/Parquet/ASpqL/src/[39m[90m[4msimple_reader.jl:27[24m[39m
     [3] [0m[1mread_parquet[22m
    [90m   @[39m [90m~/.julia/packages/Parquet/ASpqL/src/[39m[90m[4msimple_reader.jl:21[24m[39m[90m [inlined][39m
     [4] [0m[1mmacro expansion[22m
    [90m   @[39m [90m./[39m[90m[4mIn[21]:6[24m[39m[90m [inlined][39m
     [5] [0m[1m(::var"#141#threadsfor_fun#25"{var"#141#threadsfor_fun#24#26"{UnitRange{Int64}}})[22m[0m[1m([22m[90mtid[39m::[0mInt64; [90monethread[39m::[0mBool[0m

In [13]:
Threads.nthreads()

4